In [ ]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats
import os
import fiona
import rasterio
import rasterio.mask

In [ ]:
output_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files"

In [ ]:
# set building height
default_height = 20

In [ ]:
# set project crs
crs = 2926

In [ ]:
# set extent boundary
extent_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/neighborhood_extent.shp"
extent = gp.read_file(extent_path)

## TODO: create a shapefile for grasshopper
## add one blank row

In [ ]:
# floodplain extent for clipping
floodplain_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/floodplain.shp"
floodplain = gp.read_file(floodplain_path)

In [ ]:
# clip parcels to floodplain
parcel_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Building_Parcels.shp"
parcels = gp.read_file(parcel_path)
parcels.to_crs(crs, inplace = True)

print(parcels.head())
parcels['centroid_geom'] = parcels.centroid
parcels.set_geometry('centroid_geom',inplace = True)
parcels_clip = gp.overlay(parcels, floodplain, how = "intersection")

In [ ]:
parcels_merge = pd.merge(parcels_clip, parcels[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
parcels_merge.set_geometry('geometry_y', inplace = True)

In [ ]:
parcels_merge.drop(['geometry_x'], axis=1, inplace=True)

In [ ]:
parcels_merge.to_file(os.path.join(output_path, "neighborhood_parcels_v2.shp"))

In [ ]:
# clip catherine's topo
contours_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/DV_GIS Topo Data/06_GIS Data/02_Clipping Layers/01_Duwamish Valley/DV King Country Contours_clipped.shp"
contours = gp.read_file(contours_path)
contours.to_crs(crs, inplace = True)
contours_clip = gp.overlay(contours, extent, how = 'intersection')

In [21]:
contours_clip_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_contours_clip.shp"
contours_clip = gp.read_file(contours_clip_path)

In [22]:
contours_clip

,OBJECTID,ELEVATION,IDX_MINOR,IDX_20FT,IDX_40FT,IDX_50FT,IDX_100FT,TILE_ID,FIPS_CODE,Shape_Leng,boundary,geometry
0,382707.0,430.0,10.0,1.0,1.0,1.0,1.0,2304.0,033,1579.272640,2,"LINESTRING (1267406.119 191999.331, 1267406.13..."
1,382708.0,10.0,10.0,1.0,1.0,1.0,1.0,2304.0,033,124.112645,2,"LINESTRING (1275161.118 191977.858, 1275160.90..."
2,382709.0,20.0,10.0,20.0,1.0,1.0,1.0,2304.0,033,112.064917,2,"LINESTRING (1275221.118 191945.936, 1275222.43..."
3,382710.0,30.0,10.0,1.0,1.0,1.0,1.0,2304.0,033,232.107374,2,"LINESTRING (1272299.118 191932.761, 1272297.66..."
4,382711.0,200.0,10.0,100.0,200.0,100.0,100.0,2304.0,033,503.484610,2,"LINESTRING (1281839.100 191941.533, 1281839.11..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2689,629490.0,350.0,10.0,1.0,1.0,50.0,1.0,2403.0,033,9434.086045,2,"LINESTRING (1264518.146 197511.530, 1264518.11..."
2690,629491.0,360.0,10.0,20.0,40.0,1.0,1.0,2403.0,033,8532.467439,2,"LINESTRING (1264512.407 197260.427, 1264512.11..."
2691,629492.0,370.0,10.0,1.0,1.0,1.0,1.0,2403.0,033,9559.700957,2,"LINESTRING (1264503.559 196875.886, 1264503.11..."
2692,629493.0,380.0,10.0,20.0,1.0,1.0,1.0,2403.0,033,5287.163010,2,"LINESTRING (1263947.738 192442.933, 1263948.11..."


In [50]:
# if elevation > 0, take every contour 10
contours_below_SL = contours_clip[contours_clip['ELEVATION']<=20]
contours_above_SL = contours_clip[contours_clip['IDX_20FT']==20]
contours_above_SL = contours_above_SL[contours_above_SL['ELEVATION']>20]
contours_trimmed = pd.concat([contours_below_SL,contours_above_SL])

In [51]:
len(contours_trimmed), len(contours_clip)

(1322, 2694)

In [52]:
contours_trimmed.to_file(os.path.join(output_path, "neighborhood_contours_clip_test.shp"))

In [ ]:
# roads processing
roads_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
roads = gp.read_file(roads_path)
roads_clip = gp.overlay(roads, extent, how='intersection')
roads_clip.to_crs(crs, inplace = True)
roads_clip.to_file(os.path.join(output_path, "neighborhood_streets.shp"))

In [ ]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

Buildings processing
- create raster topo from contours
- subtract out contours from building heights to extrude in rhino

In [ ]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)
buildings_clip = gp.overlay(buildings, floodplain, how = "intersection")

In [ ]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
buildings_merge.set_geometry('geometry_y', inplace = True)

In [ ]:
# fill 0 and NA values to default_height
buildings_merge['BP99_APEX'].fillna(default_height, inplace = True)
buildings_merge['BP99_APEX'] = np.where(buildings_merge['BP99_APEX']==0,default_height, buildings_merge['BP99_APEX'])
buildings_merge['BP99_APEX'] = buildings_merge['BP99_APEX'].astype(int)

In [ ]:
# TODO: additional processing to make resolution a bit better

# bounding box of each building to simplify geometries

# remove buildings that are < minimum area

In [ ]:
buildings_merge.to_crs(crs, inplace = True)
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file(os.path.join(output_path,"neighborhood_buildings.shp"))

TOPO PROCESSING
- merging bathymetry data with topography dem

In [ ]:
# mask topo_rast by extent
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/topo.tif"

with fiona.open(extent_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(topo_rast) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [ ]:
topo_outpath = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_topo.asc"

# convert to ascii file
out_meta.update({"driver": "AAIGrid",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(topo_outpath, "w", **out_meta) as dest:
    dest.write(out_image)

EXTRA PROCESSING FOR ASSIGNING ABSOLUTE BUILDING HEIGHTS FROM TERRAIN

In [ ]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/min_bath_topo.tif"
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')
merged_heights['height_upd'] = np.where((merged_heights['BP99_APEX'].isna()==False) | (merged_heights['mean'].isna()==False), default_height, (merged_heights['BP99_APEX']-merged_heights['mean']))
merged_heights.set_geometry('geometry_y')
merged_heights[['OBJECTID','BP99_APEX','mean','min','max','median','std','height_upd','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/building_heights_upd.shp")